In [ ]:
import os
import sys
import math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


sys.path.append('../..')
import model.Baseline as baseline
import model.my_model as mymodel
import model.util_loss as util_ls
import model.util_dataloader as util_dl
import model.util_model as util_md
import data_process.util_data as util_dt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")

# local
file_save_dir = r'C:\Users\29492\Desktop\exp12'
file_load_dir = r'D:\dataset\DataFrames\UKDALE'
# server
# file_save_dir = r'../../data'
# file_load_dir = r'../../data'

dm = util_md.DictManager(file_save_dir)
mwm = util_md.ModelWeightManager(file_save_dir)

In [ ]:
dataframe_path = os.path.join(file_load_dir, r'house_1_filtered.csv')
df = pd.read_csv(dataframe_path)
df.set_index('time', inplace=True)


apps_name = df.columns.to_list()
print(apps_name)
apps_to_train = ['laptop-4', 'washing_machine-5', 'kettle-10', 'toaster-11', 'microwave-13']
apps_name = apps_to_train + [x for x in apps_name if x not in apps_to_train]
df = df[apps_name]
apps_name = df.columns.to_list()
print(apps_name)
data = df.to_numpy()
util_dt.print_shape(data)


total_len = len(df)
num_sample_per_day = int(60*60*24/8)
num_days = total_len/num_sample_per_day
print(num_days)


on_threshold = 30
sliding_window_len = 599
in_seq_l = sliding_window_len
out_dim = len(apps_name)
print(f'in_seq_l: {in_seq_l}; out_dim: {out_dim}')


metric_dict_one2one = {'MAE(apps)':util_ls.MAE(single_class=True),
                'SAE':util_ls.SignalAggregateError(single_class=True, period_len=450),
                'MAE(offon)':util_ls.MAE_off_on(single_class=True)}
metric_dict_one2all = {'MAE(apps)':util_ls.MAE(single_class=False),
                       'MAE(mean)':util_ls.MAE(single_class=True),
                'SAE(apps)':util_ls.SignalAggregateError(single_class=False, period_len=450),
                'SAE(mean)':util_ls.SignalAggregateError(single_class=True, period_len=450),
                'MAE(offon)':util_ls.MAE_off_on(single_class=False)}


In [ ]:
num_day_for_test = 7*2
num_day_for_val = 7*2
num_day_for_train = int(num_days) - num_day_for_test - num_day_for_val
print(num_day_for_train, num_day_for_val, num_day_for_test)

# baseline

In [ ]:
b_s = 450*20
eval_b_s = 450*20
exp_num = 3
epoch_n = 20

for app_num in range(5, 31):

    tmp_apps = data[:, 0:app_num]
    tmp_main = np.expand_dims(tmp_apps.sum(axis=-1),axis=-1)
    tmp_data = np.concatenate((tmp_apps,tmp_main),axis=1)
    norm_data, min_val, max_val = util_dt.normalization_interval(tmp_data, 0, 1)
    main = norm_data[:,-1]
    apps = norm_data[:,:-1]
    print(f'min: {min_val}; max: {max_val}')
    util_dt.print_shape(main, apps)


    loss_factor=max_val-min_val
    print(f'loss_factor:{loss_factor}')


    w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=apps, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
    w_main = np.expand_dims(w_main, axis=1)
    w_apps_sum = w_apps.sum(axis=-1)
    w_apps_sum = np.expand_dims(w_apps_sum, axis=1)
    w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
    util_dt.print_shape(w_main, w_apps, w_apps_sum, w_apps_on)


    def split_into_dataset_baseline(main, apps, start_index, end_index):
        w_main, w_apps = util_dt.generate_window_samples(input_1=main[start_index:end_index], input_2=apps[start_index:end_index], window_size_1=sliding_window_len,
                                        window_size_2=1, offset=math.floor(sliding_window_len/2))
        w_main = np.expand_dims(w_main, axis=1)
        w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
        dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                            torch.from_numpy(w_apps).to(torch.float32),
                                            torch.from_numpy(w_apps_on).to(torch.float32))
        return dataset


    subset = split_into_dataset_baseline(main, apps, -1-num_sample_per_day*num_day_for_test, -1)
    subset.describe()
    testset = util_dl.just_to_DataLoader(dataset=subset, batch_size=eval_b_s, shuffle_flag=False)
    del subset

    subset = split_into_dataset_baseline(main, apps, -1-num_sample_per_day*(num_day_for_test+num_day_for_val), -1-num_sample_per_day*num_day_for_test)
    subset.describe()
    valset = util_dl.just_to_DataLoader(dataset=subset, batch_size=eval_b_s, shuffle_flag=False)
    del subset

    subset = split_into_dataset_baseline(main, apps, 0, num_sample_per_day*num_day_for_train)
    subset.describe()
    trainset = util_dl.just_to_DataLoader(dataset=subset, batch_size=b_s, shuffle_flag=True)
    del subset


    for i in range(exp_num):

        model = mymodel.AblationMark_baseline(in_seq_l, app_num)
        score_train_vali = util_md.train_val(device, model, epoch_n, trainset, valset, loss_factor=loss_factor,
                                            return_metric_dict=metric_dict_one2all, decay_params=[5,0.5])
        score_test = util_md.train_val(device, model, valset=testset, loss_factor=loss_factor,
                                            return_metric_dict=metric_dict_one2all)
        total_record = {'train_vali':score_train_vali, 'test':score_test}
        filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='UKDALE',model='baseline',appname='all',
                                                               date=f'appnum-0111-{app_num}-{i}')
        dm.save_dict(total_record, filename)
        # mwm.save_model_weight(model, filename)

    del trainset, valset, testset


# mark

In [ ]:
b_s = 450*20
eval_b_s = 450*20
exp_num = 3
epoch_n = 50

for app_num in range(5, 31):

    tmp_apps = data[:, 0:app_num]
    tmp_main = np.expand_dims(tmp_apps.sum(axis=-1),axis=-1)
    tmp_data = np.concatenate((tmp_apps,tmp_main),axis=1)
    norm_data, min_val, max_val = util_dt.normalization_interval(tmp_data, 0, 1)
    main = norm_data[:,-1]
    apps = norm_data[:,:-1]
    print(f'min: {min_val}; max: {max_val}')
    util_dt.print_shape(main, apps)


    loss_factor=max_val-min_val
    print(f'loss_factor:{loss_factor}')


    w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=apps, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
    w_main = np.expand_dims(w_main, axis=1)
    w_apps_sum = w_apps.sum(axis=-1)
    w_apps_sum = np.expand_dims(w_apps_sum, axis=1)
    w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
    util_dt.print_shape(w_main, w_apps, w_apps_sum, w_apps_on)


    def split_into_dataset_mark(main, apps, start_index, end_index):
        w_main, w_apps = util_dt.generate_window_samples(input_1=main[start_index:end_index], input_2=apps[start_index:end_index], window_size_1=sliding_window_len,
                                        window_size_2=1, offset=math.floor(sliding_window_len/2))
        w_main = np.expand_dims(w_main, axis=1)
        w_apps_sum = w_apps.sum(axis=-1)
        w_apps_sum = np.expand_dims(w_apps_sum, axis=1)
        w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
        dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                torch.from_numpy(w_apps_on).to(torch.float32),
                                torch.from_numpy(w_apps_sum).to(torch.float32),
                                torch.from_numpy(w_apps).to(torch.float32))
        return dataset


    subset = split_into_dataset_mark(main, apps, -1-num_sample_per_day*num_day_for_test, -1)
    subset.describe()
    testset = util_dl.just_to_DataLoader(dataset=subset, batch_size=eval_b_s, shuffle_flag=False)
    del subset

    subset = split_into_dataset_mark(main, apps, -1-num_sample_per_day*(num_day_for_test+num_day_for_val), -1-num_sample_per_day*num_day_for_test)
    subset.describe()
    valset = util_dl.just_to_DataLoader(dataset=subset, batch_size=eval_b_s, shuffle_flag=False)
    del subset

    subset = split_into_dataset_mark(main, apps, 0, num_sample_per_day*num_day_for_train)
    subset.describe()
    trainset = util_dl.just_to_DataLoader(dataset=subset, batch_size=b_s, shuffle_flag=True)
    del subset


    for i in range(exp_num):

        model = mymodel.Model_final(in_seq_l, app_num)

        score_train_vali, score_train_vali_clas = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, epoch_n, trainset, valset,
                                            loss_reg_factor=loss_factor, return_metric_reg_dict=metric_dict_one2all, decay_params=[10,0.5])

        score_test, score_test_clas  = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, valset=testset, loss_reg_factor=loss_factor,
                                            return_metric_reg_dict=metric_dict_one2all)

        total_record = {'train_vali':score_train_vali, 'test':score_test,
                    'train_vali_clas':score_train_vali_clas, 'test_clas':score_test_clas}
        filename = '{dataset}_{model}_{appname}_{date}'.format(dataset='UKDALE',model='Final',appname='all',
                                                               date=f'appnum-0111-{app_num}-{i}')
        dm.save_dict(total_record, filename)
        mwm.save_model_weight(model, filename)

    del trainset, valset, testset